# Description 
This script scrapes the stoiximan webpage. It provides a dataframe of all the upcoming EPL matches that stoiximan provides together with the odds of the Betting company. It then joins them with the 'GatheredData/eplGMTStdTimeFiles/epl-2023-GMTStandardTime.csv' file and makes a 'Newfixtures.csv' file. In the end it runs all the needed scripts so that the predictor runs correctly for each round.

In [11]:
import requests 
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
import datetime

url = 'https://en.stoiximan.gr/sport/soccer/england/premier-league/1/'
page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")

In [12]:
# A function to change team names as depicted in Stoiximan in the way they are depicted in the files
def replace_names(string):
    old_names = ['Fulham FC', 'Everton FC', 'Leeds United', 'Southampton FC', 'West Ham United', 'Arsenal FC', 'AFC Bournemouth', 'Crystal Palace FC', 'Liverpool FC', 'Brighton & Hove Albion', 'Chelsea FC', 'Tottenham Hotspur', 'Brentford FC', 'Burnley FC', 'Luton Town']
    new_names = ['Fulham', 'Everton', 'Leeds', 'Southampton', 'West Ham', 'Arsenal', 'Bournemouth', 'Crystal Palace', 'Liverpool', 'Brighton', 'Chelsea', 'Tottenham', 'Brentford', 'Burnley', 'Luton']
    
    for i in range(len(old_names)):
        string = string.replace(old_names[i], new_names[i])
    return string

In [13]:
scripts = soup.find_all("script")

In [14]:
data = []
for script in scripts:
    data.extend(script)

Odds = []
Dates = []
HomeTeams = []
AwayTeams = []

data = data[3]
data = re.sub(r"window\[\"initial_state\"\]=", "", data)
print(data)
d = json.loads(data)


for match in d['data']['blocks'][0]['events']:
    teams = match['shortName'].split('-')
    HomeTeams.append(replace_names(teams[0]))
    AwayTeams.append(replace_names(teams[1]))
    Dates.append(datetime.datetime.fromtimestamp(match['startTime']/1000.0).strftime('%d/%m/%Y'))
    for x in match['markets']:
        for y in x['selections']:
            Odds.append(y['price'])
            


rows = []
while Odds:
    rows.append(Odds[:7])
    Odds = Odds[7:]

print(rows)
print(Dates)

{"data":{"leagueStandings":[],"sport":{"id":"FOOT","name":"Soccer","region":"England","url":"/sport/soccer/"},"markets":[{"id":"0","name":"Main","selected":true,"type":"Main"},{"id":"1","name":"Double chance"},{"id":"2","name":"Half Time"},{"id":"3","name":"Asian Lines"},{"id":"4","name":"Corner kicks"},{"id":"5","name":"Cards"},{"id":"6","name":"Players","type":"Players"},{"id":"7","name":"Players' specials"},{"id":"8","name":"Long term bets"}],"blocks":[{"name":"England - Premier League","shortName":"Premier League","id":"1r","url":"/sport/soccer/england/premier-league/1/","headers":[{"marketTypes":["MR12","MRES"],"columns":[{"columnSpan":1,"name":"1"},{"columnSpan":1,"name":"X"},{"columnSpan":1,"name":"2"}]},{"marketTypes":["HCTG"],"columns":[{"columnSpan":1,"name":"OVER/UNDER"}]},{"marketTypes":["BTSC"],"columns":[{"columnSpan":1,"name":"GG/NG"}]}],"layout":2,"events":[{"stats":[{"url":"https://s5.sir.sportradar.com/stoiximan/en/match/41762869","providerId":1}],"sportId":"FOOT","sh

In [15]:
df_teams = pd.DataFrame({'Home Team': HomeTeams, 'Away Team': AwayTeams})
df_values = pd.DataFrame(rows, columns=['1', 'x', '2', 'Over', 'Under', 'Yes', 'No'])
df_all = pd.concat([df_teams, df_values], axis=1)
df_all['Home Team'] = df_all['Home Team'].str.strip()
df_all['Away Team'] = df_all['Away Team'].str.strip()
display(df_all)

,Home Team,Away Team,1,x,2,Over,Under,Yes,No
0,Nottingham Forest,Sheffield United,1.90,3.55,4.15,2.18,1.70,2.00,1.75
1,Wolverhampton Wanderers,Brighton,3.60,3.75,1.98,1.75,2.07,1.65,2.15
2,Liverpool,Bournemouth,1.24,6.20,12.00,1.90,1.91,1.72,2.05
3,Fulham,Brentford,2.67,3.40,2.62,1.93,1.88,1.70,2.07
4,Tottenham,Manchester United,2.85,3.65,2.35,1.60,2.37,1.50,2.50
5,Manchester City,Newcastle United,1.72,4.05,4.55,1.72,2.12,1.70,2.07
6,Aston Villa,Everton,1.70,3.90,4.85,1.90,1.91,1.87,1.88
7,West Ham,Chelsea,3.85,3.65,1.93,1.93,1.87,1.80,1.95
8,Crystal Palace,Arsenal,5.50,4.15,1.60,1.93,1.88,1.98,1.78
9,Chelsea,Luton,1.23,6.40,12.50,2.27,1.65,2.07,1.70


In [16]:
df = pd.read_csv('GatheredData/eplGMTStdTimeFiles/epl-2023-GMTStandardTime.csv')

display(df)

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
0,1,1,11/08/2023 20:00,Turf Moor,Burnley,Man City,NaN
1,2,1,12/08/2023 12:30,Emirates Stadium,Arsenal,Nottingham Forest,NaN
2,3,1,12/08/2023 15:00,Vitality Stadium,Bournemouth,West Ham,NaN
3,4,1,12/08/2023 15:00,Amex Stadium,Brighton,Luton,NaN
4,5,1,12/08/2023 15:00,Goodison Park,Everton,Fulham,NaN
...,...,...,...,...,...,...,...
375,376,38,19/05/2024 16:00,Selhurst Park,Crystal Palace,Aston Villa,NaN
376,377,38,19/05/2024 16:00,Anfield,Liverpool,Wolves,NaN
377,378,38,19/05/2024 16:00,Kenilworth Road,Luton,Fulham,NaN
378,379,38,19/05/2024 16:00,Etihad Stadium,Man City,West Ham,NaN


In [17]:
df_merged = pd.merge(df, df_all, on=['Home Team', 'Away Team'], how='right')
df_merged['Date']=Dates
display(df_merged)
df_merged.to_csv('Newfixtures.csv')

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result,1,x,2,Over,Under,Yes,No
0,NaN,NaN,18/08/2023,NaN,Nottingham Forest,Sheffield United,NaN,1.90,3.55,4.15,2.18,1.70,2.00,1.75
1,NaN,NaN,19/08/2023,NaN,Wolverhampton Wanderers,Brighton,NaN,3.60,3.75,1.98,1.75,2.07,1.65,2.15
2,14.0,2.0,19/08/2023,Anfield,Liverpool,Bournemouth,NaN,1.24,6.20,12.00,1.90,1.91,1.72,2.05
3,13.0,2.0,19/08/2023,Craven Cottage,Fulham,Brentford,NaN,2.67,3.40,2.62,1.93,1.88,1.70,2.07
4,NaN,NaN,19/08/2023,NaN,Tottenham,Manchester United,NaN,2.85,3.65,2.35,1.60,2.37,1.50,2.50
5,NaN,NaN,19/08/2023,NaN,Manchester City,Newcastle United,NaN,1.72,4.05,4.55,1.72,2.12,1.70,2.07
6,11.0,2.0,20/08/2023,Villa Park,Aston Villa,Everton,NaN,1.70,3.90,4.85,1.90,1.91,1.87,1.88
7,19.0,2.0,20/08/2023,London Stadium,West Ham,Chelsea,NaN,3.85,3.65,1.93,1.93,1.87,1.80,1.95
8,12.0,2.0,21/08/2023,Selhurst Park,Crystal Palace,Arsenal,NaN,5.50,4.15,1.60,1.93,1.88,1.98,1.78
9,26.0,3.0,25/08/2023,Stamford Bridge,Chelsea,Luton,NaN,1.23,6.40,12.50,2.27,1.65,2.07,1.70


In [18]:
#Downloading the updated results
import os
import requests

# URL of the file to download
url = "https://www.football-data.co.uk/mmz4281/2324/E0.csv"

# Name for the downloaded file
file_name = "E0-2023.csv"

# Path to the directory where you want to save the file
output_directory = "GatheredData/ExFiles"

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Full path for the downloaded file
output_path = os.path.join(output_directory, file_name)

# Download the file
response = requests.get(url)
if response.status_code == 200:
    with open(output_path, "wb") as file:
        file.write(response.content)
    print(f"File downloaded and saved as {output_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded and saved as GatheredData/ExFiles/E0-2023.csv


In [19]:
#Changing team names
temp_pd = pd.read_csv('GatheredData/ExFiles/E0-2023.csv')
team_dict = {
                     'Man City': 'Manchester City',
                     'Man United': 'Manchester United',
                     'Nott\'m Forest': 'Nottingham Forest',
                     'Newcastle': 'Newcastle United',
                     'Wolves': 'Wolverhampton Wanderers'
                         }
temp_pd['HomeTeam'] = temp_pd['HomeTeam'].replace(team_dict)
temp_pd['AwayTeam'] = temp_pd['AwayTeam'].replace(team_dict)

temp_pd.to_csv('GatheredData/ExFiles/E0-2023.csv')

In [20]:
# run the script named "my_script.py" with argument "arg1"
os.system("python NewFixturesScraping2.py")
os.system("python NewfixturesScraping2Month.py")
os.system("python NewfixturesScraping2Month3.py")
os.system("python Gather2023Season.py")
os.system("python Gather2023SeasonForm.py")
os.system("python Gather2023SeasonForm_3Month.py")

18/08/2023
0    Nottingham Forest
Name: Home Team, dtype: object
0    Sheffield United
Name: Away Team, dtype: object


/home/nickpad/Documents/EnglishPremierLeaguePredictor/NewFixturesScraping2.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EnglishPremierLeaguePredictor/NewFixturesScraping2.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(


19/08/2023
1    Wolverhampton Wanderers
2                  Liverpool
3                     Fulham
4                  Tottenham
5            Manchester City
Name: Home Team, dtype: object
1             Brighton
2          Bournemouth
3            Brentford
4    Manchester United
5     Newcastle United
Name: Away Team, dtype: object


/home/nickpad/Documents/EnglishPremierLeaguePredictor/NewFixturesScraping2.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EnglishPremierLeaguePredictor/NewFixturesScraping2.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(


20/08/2023
6    Aston Villa
7       West Ham
Name: Home Team, dtype: object
6    Everton
7    Chelsea
Name: Away Team, dtype: object


...

7    Brentford
8      Chelsea
Name: HomeTeam, dtype: object
7    Tottenham
8    Liverpool
Name: AwayTeam, dtype: object
0        Team  M  G GA PTS xG xGA PPDA OPPDA  DC  ODC  PTS/M
14  Brentford  0  0  0   0  0   0    0     0   0    0      0
9     Chelsea  0  0  0   0  0   0    0     0   0    0      0
0        Team  M  G GA PTS xG xGA PPDA OPPDA  DC  ODC  PTS/M
10  Tottenham  0  0  0   0  0   0    0     0   0    0      0
11  Liverpool  0  0  0   0  0   0    0     0   0    0      0
14/08/2023
2023-08-13
2023-08-13


/home/nickpad/Documents/EnglishPremierLeaguePredictor/Gather2023SeasonForm_3Month.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EnglishPremierLeaguePredictor/Gather2023SeasonForm_3Month.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(


9    Manchester United
Name: HomeTeam, dtype: object
9    Wolves
Name: AwayTeam, dtype: object
0                Team  M  G GA PTS xG xGA PPDA OPPDA  DC  ODC  PTS/M
13  Manchester United  0  0  0   0  0   0    0     0   0    0      0
Empty DataFrame
Columns: [Team, M, G, GA, PTS, xG, xGA, PPDA, OPPDA, DC, ODC, PTS/M]
Index: []
               HTeam  HM  HG  HGA  HPTS  ...  HPPDA  HOPPDA  HDC  HODC  HPTS/M
0            Burnley   0   0    0     0  ...      0       0    0     0       0
1            Arsenal   0   0    0     0  ...      0       0    0     0       0
2        Bournemouth   0   0    0     0  ...      0       0    0     0       0
3           Brighton   0   0    0     0  ...      0       0    0     0       0
4            Everton   0   0    0     0  ...      0       0    0     0       0
5   Sheffield United   0   0    0     0  ...      0       0    0     0       0
6   Newcastle United   0   0    0     0  ...      0       0    0     0       0
7          Brentford   0   0    0     0 

/home/nickpad/Documents/EnglishPremierLeaguePredictor/Gather2023SeasonForm_3Month.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(


0